In [1]:
%cd ..\..\..
import pandas as pd
import gc

D:\Projects\CondaAnalyses\PMI


In [2]:
# import the base df
data_path = 'data_checkpoints/usecase_cab/'
df = pd.read_csv(data_path+'base_df.csv', index_col=0)
df['time']= pd.to_datetime(df['time'])
df.head()

,id,file_name,latitude,longitude,time,occupancy
0,abboip_1211033530,abboip,37.75153,-122.39447,2008-05-17 14:12:10,0
1,abboip_1211033614,abboip,37.75149,-122.39447,2008-05-17 14:13:34,0
2,abboip_1211033674,abboip,37.75149,-122.39447,2008-05-17 14:14:34,0
3,abboip_1211033735,abboip,37.75149,-122.39446,2008-05-17 14:15:35,0
4,abboip_1211035303,abboip,37.75144,-122.39449,2008-05-17 14:41:43,0


In [3]:
# drop columns not needed for the occupancy_state_change
df.drop(['latitude', 'longitude'], axis=1, inplace=True)
df.head(2)

,id,file_name,time,occupancy
0,abboip_1211033530,abboip,2008-05-17 14:12:10,0
1,abboip_1211033614,abboip,2008-05-17 14:13:34,0


In [4]:
df['occupancy_state_change'] = df.sort_values(['time']).groupby(by=['file_name'])['occupancy'].diff()
df = df.fillna(1)

df['occupancy_state_change'] = df['occupancy_state_change'].abs()
df['id_event'] = df['file_name'] + '_' + df['time'].astype(str)
df['id_event'] = df['id'][df['occupancy_state_change'] == 1.0]

df = df.fillna(method="ffill")
df.head(2)

,id,file_name,time,occupancy,occupancy_state_change,id_event
0,abboip_1211033530,abboip,2008-05-17 14:12:10,0,1.0,abboip_1211033530
1,abboip_1211033614,abboip,2008-05-17 14:13:34,0,0.0,abboip_1211033530


In [5]:
# drop additional columns to keep dataframe lean
df.drop(['file_name', 'time', 'occupancy', 'occupancy_state_change'], axis=1, inplace=True)
df.head(2)

,id,id_event
0,abboip_1211033530,abboip_1211033530
1,abboip_1211033614,abboip_1211033530


In [6]:
df.to_csv(data_path+'occupancy_state_change.csv')
del df
gc.collect()

0